In [1]:
import pandas as pd
import numpy as np

from scipy.spatial.distance import jaccard, pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet

import matplotlib.pyplot as plt

In [2]:
# load the data: rows are titles and columns are review periodicals
df = pd.read_csv('../data/processed/books1965-1984_10-15.tsv', sep='\t', index_col=0)
df

,AB Bookman's Weekly,Publishers Weekly,Esquire,Booklist,Journal of Aesthetics and Art Criticism,International Philosophical Quarterly,Journal of Marketing,Harvard Law Review,Journal of Business Education,Journal of Home Economics,...,Black Warrior Review,Computers and the Humanities,American Arts,Essays on Canadian Writing`,Performing Arts Review,"Journal of Arts Management, Law, and Society","Studio International, Review",Journal of Black Studies,Lone Star Review,Aspen Journal of the Arts
"Classics In The Kitchen || AABERG, Jean",2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Complete Book Of Knitting || ABBEY, Barbara",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"A Locomotive Engineer's Album || ABDILL, George B",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"House Of India Cookbook || ABDULLAH, Syed",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Rockefeller Billions: World's Most Stupendous Fortune || ABELS, Jules",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"The Structuring Of Organizations || MINTZBERG, Henry",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Emergence Of The American University || VEYSEY, Laurence",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Removing Obstacles To Economic Growth || WACHTER, Michael L",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Policy Perspectives || WRISTON, Henry M",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# preprocess: binarize the data (for jaccard similarity) and limit to magazines with at least 5 reviews
#df[:] = np.where(df==0, 0, 1)
my_cols = df.columns[df.sum() >= 50]
df = df[my_cols]
df

,AB Bookman's Weekly,Publishers Weekly,Esquire,Booklist,Journal of Aesthetics and Art Criticism,International Philosophical Quarterly,Journal of Marketing,Harvard Law Review,Journal of Business Education,Journal of Home Economics,...,Drexel Library Quarterly,Library Resources and Technical Services,American Antiquity,Religious Studies,Dance News,Daedalus,Theological Studies,Biblical Theology Bulletin,Educational Forum,Threepenny Review
"Classics In The Kitchen || AABERG, Jean",2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Complete Book Of Knitting || ABBEY, Barbara",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"A Locomotive Engineer's Album || ABDILL, George B",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"House Of India Cookbook || ABDULLAH, Syed",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Rockefeller Billions: World's Most Stupendous Fortune || ABELS, Jules",1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"The Structuring Of Organizations || MINTZBERG, Henry",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"The Emergence Of The American University || VEYSEY, Laurence",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Removing Obstacles To Economic Growth || WACHTER, Michael L",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Policy Perspectives || WRISTON, Henry M",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# construct a similarity matrix comparing each journal to each other, using jaccard similarity
dists = pdist(df.T, metric='cosine')
square_dists = squareform(dists)
square_dists = pd.DataFrame(square_dists, index=df.columns, columns=df.columns)
square_dists

,AB Bookman's Weekly,Publishers Weekly,Esquire,Booklist,Journal of Aesthetics and Art Criticism,International Philosophical Quarterly,Journal of Marketing,Harvard Law Review,Journal of Business Education,Journal of Home Economics,...,Drexel Library Quarterly,Library Resources and Technical Services,American Antiquity,Religious Studies,Dance News,Daedalus,Theological Studies,Biblical Theology Bulletin,Educational Forum,Threepenny Review
AB Bookman's Weekly,0.000000,0.942976,0.987363,0.934794,0.978612,1.000000,0.997874,0.998513,1.000000,0.994280,...,0.984022,0.996600,0.998363,1.000000,0.993281,0.998197,1.000000,1.000000,1.000000,0.998197
Publishers Weekly,0.942976,0.000000,0.910372,0.591494,0.990499,0.998808,0.999154,0.983033,0.999542,0.990134,...,0.998728,1.000000,1.000000,0.997684,0.986181,0.992346,1.000000,1.000000,0.997289,0.985171
Esquire,0.987363,0.910372,0.000000,0.943017,0.996264,1.000000,1.000000,0.994026,1.000000,0.997447,...,1.000000,1.000000,1.000000,1.000000,0.991002,1.000000,1.000000,1.000000,1.000000,0.995172
Booklist,0.934794,0.591494,0.943017,0.000000,0.990655,0.999711,0.996439,0.986016,0.997776,0.983543,...,0.990942,0.992992,0.999156,0.998073,0.989179,0.993962,0.997761,0.998973,0.994736,0.993497
Journal of Aesthetics and Art Criticism,0.978612,0.990499,0.996264,0.990655,0.000000,0.990260,1.000000,0.998388,1.000000,1.000000,...,0.996536,1.000000,1.000000,1.000000,0.996358,1.000000,1.000000,1.000000,1.000000,0.996092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Daedalus,0.998197,0.992346,1.000000,0.993962,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
Theological Studies,1.000000,1.000000,1.000000,0.997761,1.000000,0.981796,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.969692,1.000000,1.000000,0.000000,0.983850,1.000000,1.000000
Biblical Theology Bulletin,1.000000,1.000000,1.000000,0.998973,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.983850,0.000000,1.000000,1.000000
Educational Forum,1.000000,0.997289,1.000000,0.994736,1.000000,1.000000,1.000000,1.000000,1.000000,0.992433,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000


In [5]:
Z = linkage(dists, 'weighted')
c, coph_dists = cophenet(Z, dists)
print(Z.shape)
print(c)

(398, 4)
0.7747587962097044


In [6]:
fig = plt.figure(figsize=(40, 400))
ax = fig.add_subplot(1, 1, 1)
labels = [f'{col}: {df[col].sum()}' for col in df.columns]
dn = dendrogram(Z, 
                ax=ax, 
                labels=labels, 
                color_threshold=0.95*max(Z[:,2]), 
                orientation='right',
                distance_sort='descending'
)

ax.tick_params(axis='x', which='major', labelsize=40)
ax.tick_params(axis='y', which='major', labelsize=20)
ax.set_xlim(xmin=.5)


plt.show()

In [31]:
[f'{col}: {df[col].sum()}' for col in df.columns]

["AB Bookman's Weekly: 4349",
 'Publishers Weekly: 51001',
 'Esquire: 620',
 'Booklist: 60419',
 'Journal of Aesthetics and Art Criticism: 954',
 'International Philosophical Quarterly: 164',
 'Journal of Marketing: 573',
 'Harvard Law Review: 382',
 'Journal of Business Education: 506',
 'Journal of Home Economics: 224',
 'Social Education: 1446',
 'Library Journal: 77369',
 'Small Press Review: 201',
 'World Literature Today: 6548',
 'American Historical Review: 14637',
 'Encounter: 2300',
 'Geographical Journal: 3130',
 'Journal of Historical Geography: 383',
 'New Statesman: 11774',
 'Times Literary Supplement: 50327',
 'Bibliographical Society of America, Papers: 1634',
 'International Labour Review: 1822',
 "Center for Children's Books, Bulletin: 10372",
 'Christian Science Monitor: 9077',
 'Horn Book Magazine: 6513',
 'Kirkus Reviews: 54167',
 'Saturday Review/World: 434',
 'Top of the News: 1036',
 "Children's Book Review Service: 3429",
 'Childhood Education: 3413',
 'Catholic